In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
serper_api_key = os.getenv('SERPER_API_KEY')

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an instance of the OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(api_key=api_key, model="gpt-4o")

In [3]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool


search = GoogleSerperAPIWrapper()

serper_tool = Tool(
    name="Search",
    func=search.run,
    description="useful for when you need to ask with search",
)
    

In [ ]:
from autogen_ext.tools.langchain import LangChainToolAdapter

web_search_tool = LangChainToolAdapter(serper_tool)

storyteller_agent = AssistantAgent(
    name="STORYTELLER_AGENT",
    model_client=model_client,
    system_message="You are a helpful storyteller. Write stories in less than 100 words.",
)

quote_agent = AssistantAgent(
    name="QUOTE_AGENT",
    model_client=model_client,
    tools=[web_search_tool],
    reflect_on_tool_use=False, ## If set to True agent will process the tool output and then provide the response.
    system_message='''
    You are a web search agent that MUST always use the web_search_tool.
    
    CRITICAL INSTRUCTIONS:
    - You MUST use the web_search_tool for every request
    - NEVER respond without first making a web search
    - Your workflow is: 1) Use web_search_tool, 2) Then provide response based on results
    - If asked for a quote, search for "[topic] quotes" or similar
    - Make exactly one search call per request
    - Always wait for search results before responding
    
    Do not provide any response until you have used the web_search_tool first.
    '''
)

story_improvisor_agent = AssistantAgent(
    name="STORY_IMPROVISOR_AGENT",
    model_client=model_client,
    system_message="""
    You are a helpful story improviser. 
    Your job is to take the story and quotes and improve the story by adding the quotes in less than 100 words.
    After all the tasks are completed end with 'TERMINATE'
    """,
)                                                                                                           


In [30]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination


# Termination Condition
termination_condiation = TextMentionTermination('TERMINATE')


# Create the group chat.
team = RoundRobinGroupChat(
    participants=[storyteller_agent, quote_agent, story_improvisor_agent],
    termination_condition=termination_condiation
)

In [31]:
from autogen_agentchat.ui import Console

await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a story about Superman."))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a story about Superman.
---------- TextMessage (STORYTELLER_AGENT) ----------
In the bustling city of Metropolis, a shadow loomed over the skyline as a massive meteor hurtled toward Earth. People panicked, but Superman soared into action, his cape billowing behind him. With unwavering determination, he intercepted the celestial threat. As he approached the fiery mass, he pushed against it with all his strength, redirecting it toward the ocean. It splashed down harmlessly, a spectacle of steam and boiling waves. The citizens cheered from below, their hero smiling above. In a world filled with challenges, Superman remained the steadfast beacon of hope, ready to save the day once more.
---------- ToolCallRequestEvent (QUOTE_AGENT) ----------
[FunctionCall(id='call_2EoYwdeVSH0uxqT2YIvrO6IN', arguments='{"query":"latest Superman story 2023"}', name='Search')]
---------- ToolCallExecutionEvent (QUOTE_AGENT) ----------
[FunctionExecutionResult(co

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a story about Superman.', type='TextMessage'), TextMessage(source='STORYTELLER_AGENT', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=124), metadata={}, content='In the bustling city of Metropolis, a shadow loomed over the skyline as a massive meteor hurtled toward Earth. People panicked, but Superman soared into action, his cape billowing behind him. With unwavering determination, he intercepted the celestial threat. As he approached the fiery mass, he pushed against it with all his strength, redirecting it toward the ocean. It splashed down harmlessly, a spectacle of steam and boiling waves. The citizens cheered from below, their hero smiling above. In a world filled with challenges, Superman remained the steadfast beacon of hope, ready to save the day once more.', type='TextMessage'), ToolCallRequestEvent(source='QUOTE_AGENT', models_usage=RequestUsage(prompt_tokens=317, com